In [1]:
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
label_encode = LabelEncoder()
data_dict = {}
name = ["1", "2", "3", "4","5", "6", "7", "8","9", "10", "11", "12","13", "14", "15", "16","17", "18","19", "activity"]
data_list = []
for i in range(1, 21):
    x = 18
    if i ==16:
        x = 14
    folder = '%02ddata' % i
    data_dict[folder] = {}
    for j in range(1, x):
        filename = '{}.csv'.format(j)
        filepath = os.path.join(folder, filename)
        #data_dict[folder][j] = pd.read_csv(filepath,names=name)
        #data_dict[folder][j]["activityEncode"] = label_encode.fit_transform(data_dict[folder][j]["activity"].values.ravel())
        data = pd.read_csv(filepath,names=name)
        data['user-id'] = [i] * len(data)
        data_list.append(data)


In [2]:
combined_data = pd.concat(data_list, axis=0, ignore_index=True)

In [3]:
combined_data

,1,2,3,4,5,6,7,8,9,10,...,12,13,14,15,16,17,18,19,activity,user-id
0,38658,32423,32763,33863,33180,33154,37216,34168,34272,32996,...,35516,27596,28400,29768,28008,28816,28468,28256,walk,1
1,37928,32173,32973,33409,32926,33154,37216,34168,34272,32996,...,35516,27596,28400,29768,28008,28816,28468,28256,walk,1
2,34765,32093,32946,34851,32397,33154,37216,34168,34272,32996,...,35516,27596,28400,29768,28008,28816,28468,28256,walk,1
3,36068,32653,32843,34061,32267,33154,37216,34168,34272,32996,...,35516,27596,28400,29768,28008,28816,28468,28256,walk,1
4,32098,33146,33192,34309,32484,33154,37216,34168,34272,32996,...,35516,27596,28400,29768,28008,28816,28468,28256,walk,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21580893,33042,32887,32761,32553,32391,34600,36948,33170,33808,33592,...,35796,28802,27376,29400,28280,32132,28597,29249,stand,20
21580894,32854,32704,32671,32573,32632,34600,36948,33170,33808,33592,...,35796,28802,27376,29400,28280,32132,28597,29249,stand,20
21580895,32514,32541,32698,32693,34241,34600,36948,33170,33808,33592,...,35796,28802,27376,29400,28280,32132,28597,29249,stand,20
21580896,32455,32603,32674,32558,32206,34600,36948,33170,33808,33592,...,35796,28802,27376,29400,28280,32132,28597,29249,stand,20


In [4]:
from pandas import read_csv, unique

import numpy as np

from scipy.interpolate import interp1d
from scipy.stats import mode

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

from tensorflow import stack
#from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling1D, BatchNormalization, MaxPool1D, Reshape, Activation
from keras.layers import Conv1D, LSTM
from keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")


In [5]:
label_encode = LabelEncoder()
combined_data['activityEncode'] = label_encode.fit_transform(combined_data['activity'].values.ravel())
combined_data

,1,2,3,4,5,6,7,8,9,10,...,13,14,15,16,17,18,19,activity,user-id,activityEncode
0,38658,32423,32763,33863,33180,33154,37216,34168,34272,32996,...,27596,28400,29768,28008,28816,28468,28256,walk,1,17
1,37928,32173,32973,33409,32926,33154,37216,34168,34272,32996,...,27596,28400,29768,28008,28816,28468,28256,walk,1,17
2,34765,32093,32946,34851,32397,33154,37216,34168,34272,32996,...,27596,28400,29768,28008,28816,28468,28256,walk,1,17
3,36068,32653,32843,34061,32267,33154,37216,34168,34272,32996,...,27596,28400,29768,28008,28816,28468,28256,walk,1,17
4,32098,33146,33192,34309,32484,33154,37216,34168,34272,32996,...,27596,28400,29768,28008,28816,28468,28256,walk,1,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21580893,33042,32887,32761,32553,32391,34600,36948,33170,33808,33592,...,28802,27376,29400,28280,32132,28597,29249,stand,20,11
21580894,32854,32704,32671,32573,32632,34600,36948,33170,33808,33592,...,28802,27376,29400,28280,32132,28597,29249,stand,20,11
21580895,32514,32541,32698,32693,34241,34600,36948,33170,33808,33592,...,28802,27376,29400,28280,32132,28597,29249,stand,20,11
21580896,32455,32603,32674,32558,32206,34600,36948,33170,33808,33592,...,28802,27376,29400,28280,32132,28597,29249,stand,20,11


In [6]:
df_test = combined_data[combined_data['user-id'] > 16]
df_train = combined_data[combined_data['user-id'] <= 16]

In [7]:
for x in range(1,20):
    x = str(x)
    df_train[x] = (df_train[x]-df_train[x].min())/(df_train[x].max()-df_train[x].min())

df_train

,1,2,3,4,5,6,7,8,9,10,...,13,14,15,16,17,18,19,activity,user-id,activityEncode
0,0.589883,0.494743,0.499931,0.516716,0.506294,0.588358,0.590938,0.538290,0.620165,0.496135,...,0.715279,0.433335,0.454265,0.426763,0.439652,0.434349,0.431008,walk,1,17
1,0.578744,0.490929,0.503136,0.509789,0.502419,0.588358,0.590938,0.538290,0.620165,0.496135,...,0.715279,0.433335,0.454265,0.426763,0.439652,0.434349,0.431008,walk,1,17
2,0.530480,0.489708,0.502724,0.531792,0.494347,0.588358,0.590938,0.538290,0.620165,0.496135,...,0.715279,0.433335,0.454265,0.426763,0.439652,0.434349,0.431008,walk,1,17
3,0.550362,0.498253,0.501152,0.519738,0.492363,0.588358,0.590938,0.538290,0.620165,0.496135,...,0.715279,0.433335,0.454265,0.426763,0.439652,0.434349,0.431008,walk,1,17
4,0.489784,0.505776,0.506477,0.523522,0.495674,0.588358,0.590938,0.538290,0.620165,0.496135,...,0.715279,0.433335,0.454265,0.426763,0.439652,0.434349,0.431008,walk,1,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17144612,0.501579,0.501030,0.487739,0.500999,0.490043,0.588536,0.585078,0.547143,0.730659,0.463551,...,0.742725,0.499542,0.415749,0.441317,0.480128,0.382295,0.440241,walk-downstairs,16,20
17144613,0.501839,0.503960,0.495262,0.499443,0.513558,0.588536,0.585078,0.547143,0.730659,0.463551,...,0.742725,0.499542,0.415749,0.441317,0.480128,0.382295,0.440241,walk-downstairs,16,20
17144614,0.500771,0.500481,0.499657,0.502190,0.524269,0.588536,0.585078,0.547143,0.730659,0.463551,...,0.742725,0.499542,0.415749,0.441317,0.480128,0.382295,0.440241,walk-downstairs,16,20
17144615,0.501595,0.500374,0.503426,0.502312,0.520211,0.588536,0.585078,0.547143,0.730659,0.463551,...,0.742725,0.499542,0.415749,0.441317,0.480128,0.382295,0.440241,walk-downstairs,16,20


In [12]:
def segments(df, time_steps, step, label_name):
    N_FEATURES = 19
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
        for x in range(1,20):
            x = df[str(x)].values[i:i+time_steps]
            segments.append(x)

        label = mode(df[label_name][i:i+time_steps])[0][0]
        labels.append(label)

    reshaped_segments = np.asarray(segments, dtype=np.float32).reshape(-1, time_steps, N_FEATURES)
    labels = np.asarray(labels)

    return reshaped_segments, labels

TIME_PERIOD = 2000
STEP_DISTANCE = 2000
LABEL = 'activityEncode'
x_data, y_data = segments(combined_data, TIME_PERIOD, STEP_DISTANCE, LABEL)

In [13]:
print('x_train shape:', x_data.shape)
print('Training samples:', x_data.shape[0])
print('y_train shape:', y_data.shape)



x_train shape: (10790, 2000, 19)
Training samples: 10790
y_train shape: (10790,)


In [14]:
np.save('CLS-data2000.npy', x_data)
np.save('CLS-label2000.npy',y_data)


In [ ]:
import numpy as np
x=np.load('CLS-Train_label.npy')

In [ ]:
print(x.shape)